# Linear Programming Tube Cutting
Using PuLP for linear programming in python. Solving a basic material optimization problem. Determine which parts should be cut on which piece of stock such that scrap material is minimized. All parts and stock are tubes which only have a length. This problem is similar to a 1d bin packing problem.

- $ l_i $ = Length of part i. i = 1, 2, 3, ... n
- $ M_j $ = Length of stock j. j = 1, 2, 3, ... m
- $ x_{i,j} $ = {1 if part i is cut on stock j, 0 otherwise}
- $ y_j $ = {1 if stock j is used, 0 otherwise}

Objective:
- Minimize the excess stock material after all parts are cut.

$$ min \sum_{j=1}^{m} (M_j * y_j - \sum_{i=1}^{n} l_i * x_{i,j}) $$

Subject to:
- Total length of parts cut can not exceed stock length. Implies stock must be used if any parts are to be cut on it.
- All parts must be cut once

$$ M_j * y_j - \sum_{i=1}^{n} x_{i,j} \geq 0 \quad \forall j$$
$$ \sum_{j=1}^{m} x_{i,j} = 1 \quad \forall i$$

In [41]:
import pulp
import pandas as pd

In [42]:
# Test data
# part_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= "Test Part Data")
# stock_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= "Test Stock Data")

# .049 data
part_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".049 Part Data")
stock_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".049 Stock Data")

# # .065 data
# # Part Number, Quantity, Description, Length
# # Quantity is unused?? should remove it??
# part_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".065 Part Data")

# # Stock Number, Length
# stock_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".065 Stock Data")

In [43]:
prob = pulp.LpProblem("Tube_Cut", pulp.LpMinimize)

In [44]:
# Converts part and stock number to lists
# Could use index numbers instead? It seems implied that part/stock numbers must be unique
# These values are used for i and j. Probably should just become index values
part_list = part_df['Part Number'].to_list()
stock_list = stock_df['Stock Number'].to_list()

In [45]:
# # All possible pairs between parts and stock
# Is there a easier way to do this? It isn't too bad
part_on_stock = [(i, j) for i in part_list for j in stock_list]

In [46]:
# Dict of each part number and it's length
part_lengths = dict(zip(part_list, part_df["Length"]))
stock_lengths = dict(zip(stock_list, stock_df["Length"]))

In [47]:
# variables in dictionary form, indexed by values in part_list, stock_list
# upper and lower bounds of variables are added as constraints
part_vars = pulp.LpVariable.dicts("Part_Stock", (part_list, stock_list), 0, 1, cat= pulp.LpInteger)
stock_vars = pulp.LpVariable.dicts("Stock", stock_list, 0, 1, cat= pulp.LpInteger)

In [48]:
# It seems like the first thing added to the problem with no constraint given, is assumed to be the objective
# Minimize the difference between the stock length and length of tubes cut on the piece of stock
prob += (pulp.lpSum([stock_lengths[j] * stock_vars[j] - 
                    pulp.lpSum([part_lengths[i] * part_vars[i][j] for i in part_list]) 
                    for j in stock_list]), 
                    "Excess Stock")

In [49]:
# Requires the tubes cut on stock to not exceed the tube length
# Not sure why I used the <= 0 here
for j in stock_list:
    prob += (pulp.lpSum([part_lengths[i] * part_vars[i][j] for i in part_list]) - stock_lengths[j] * stock_vars[j] <= 0, 
             ("Stock Capacity " + str(j)))

In [50]:
# All parts must only be cut once on a tube
for i in part_list:
    prob += (pulp.lpSum([part_vars[i][j] for j in stock_list]) - 1 == 0, 
             ("Part Cut " + str(i)))

In [51]:
print(prob)
prob.solve()
# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)
part_on_stock = [v.varValue for v in prob.variables()]

# The optimised objective function value is printed to the console
print("Excess Material = ", pulp.value(prob.objective))


Tube_Cut:
MINIMIZE
-30.59*Part_Stock_19_1 + -30.59*Part_Stock_19_2 + -30.59*Part_Stock_19_3 + -30.59*Part_Stock_19_4 + -30.59*Part_Stock_19_5 + -30.59*Part_Stock_19_6 + -30.59*Part_Stock_19_7 + -30.59*Part_Stock_19_8 + -30.59*Part_Stock_20_1 + -30.59*Part_Stock_20_2 + -30.59*Part_Stock_20_3 + -30.59*Part_Stock_20_4 + -30.59*Part_Stock_20_5 + -30.59*Part_Stock_20_6 + -30.59*Part_Stock_20_7 + -30.59*Part_Stock_20_8 + -14.9*Part_Stock_35_1 + -14.9*Part_Stock_35_2 + -14.9*Part_Stock_35_3 + -14.9*Part_Stock_35_4 + -14.9*Part_Stock_35_5 + -14.9*Part_Stock_35_6 + -14.9*Part_Stock_35_7 + -14.9*Part_Stock_35_8 + -14.9*Part_Stock_36_1 + -14.9*Part_Stock_36_2 + -14.9*Part_Stock_36_3 + -14.9*Part_Stock_36_4 + -14.9*Part_Stock_36_5 + -14.9*Part_Stock_36_6 + -14.9*Part_Stock_36_7 + -14.9*Part_Stock_36_8 + -13.05*Part_Stock_43_1 + -13.05*Part_Stock_43_2 + -13.05*Part_Stock_43_3 + -13.05*Part_Stock_43_4 + -13.05*Part_Stock_43_5 + -13.05*Part_Stock_43_6 + -13.05*Part_Stock_43_7 + -13.05*Part_Stock_43_8

In [52]:
# part_on_stock
# print(part_on_stock)
# print(part_vars)
# prob.variables()[0]

for j in stock_list:
    print(j)
    for i in part_list:
        if part_vars[i][j].varValue != 0:
            print(str(part_vars[i][j]) + ": " + str(part_vars[i][j].varValue))

1
2
Part_Stock_36_2: 1.0
Part_Stock_44_2: 1.0
Part_Stock_20_2: 1.0
Part_Stock_50_2: 1.0
3
Part_Stock_43_3: 1.0
Part_Stock_19_3: 1.0
Part_Stock_67_3: 1.0
4
5
Part_Stock_35_5: 1.0
Part_Stock_48_5: 1.0
Part_Stock_68_5: 1.0
6
7
Part_Stock_66_7: 1.0
Part_Stock_47_7: 1.0
Part_Stock_49_7: 1.0
Part_Stock_72_7: 1.0
8
